# COGS 189: EEG-Based Brain Computer Interfaces
## Assignment 2: P300 Speller Classification via Linear Discriminant Analysis
Created By: Alessandro "Ollie" D'Amico
***
**Assigned: 2/12/2019** <br>
**Due: 2/20/2019 at 11:59 PM**
***
## Overview:
For this assignment, we will be working with a BCI competition dataset, with the primary goal of classifying target vs non-target trials from a P300 speller. The data we will be using comes from the 2nd Berlin BCI Competition, and was provided by Wadsworth Center, NYS Department of Health (Jonathan R. Wolpaw). Please go to the following link: 
http://www.bbci.de/competition/ii/
<br><br>
We will be working with **Dataset IIb**. Please click on the description of this dataset (pdf format is superior) and read through the page in order to answer the following questions
Here is a direct link: http://www.bbci.de/competition/ii/albany_desc/albany_desc_ii.pdf
***
***Question 1:*** What is the *dt* for the row/column intensification? 
<br>*(Hint: Remember that from our last assignment, the dt at 1000 Hz = 1ms, 500 Hz = 2ms, 200 Hz = 5ms)*<br>
***Question 2:*** How long (in ms) was each row/column intensified?<br>
***Question 3:*** How much time (in ms) was there between each row/column intensification?<br>
***Question 4:*** What is the sampling rate of the EEG data?<br>
***Question 5:*** How many times was each character intensified?
***
<img src="P300Matrix.gif">

*The .gif above is what the actual task looks like. I created the animation following the description of the dataset. As you can tell, these stimuli are being presented in rapid succession. The participant is instructed to only focus on one character at a time, until they are instructed by the task to look at a new character.*
***
The owners of this data provided us with useful descriptions of the data, along with some matlab code. We will use their documentation to epoch the data so that we can train our classifier. We will be using linear discriminant analysis (LDA) for this problem, as it performs very well on P300 paradigms. Furthermore, this algorithm is rather intuitive. We will be using sklearn as it will make this assignment easier to follow.

## Section 1: Setup
Before we can begin processing our data, we must first import some useful packages. <br>
- **numpy** is used widely for processing numerical data, and supports matrix operations natively. We will be using NumPy arrays to store our data.
- **scipy** contains useful functions to calculate filter coefficients and carry out filtering. We will be using SciPy filters to clean our EEG data.
- **matplotlib** is used to create plots. We will use it to visually explore our filter and EEG data.
- **sklearn** used for machine learning. We will use this to train our classifier and make predictions

In [ ]:
import numpy as np                                      # for dealing with data
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt                         # for plotting
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import cross_val_score

## Section 2: Pre-processing
Unfortunately, the data provided to us are in an inconvenient format. The authors saved their entire MATLAB workspace into a .mat for each session. I could not find a way to load in all of these variables into Python. I wrote code to export all of these variables independently in a format we could bring in to Python, but the result was hundreds of files coming in at over 1GB. These data would have been inconvenient to work with in Python, and so I did the pre-processing in MATLAB. The code I wrote is available on this class's GitHub, for those who are curious.
<br><br>
Although the pre-processing was conducted in MATLAB, it is important to know what was done to the data, otherwise this assignment wouldn't make any sense.
<br><br>
**Step 1: Design a filter**<br>
The first thing I did, after creating some variables to store our sampling rate, dt, and number of channels, was to create a filter. I used the filter design from the previous assignment, a 0.1 Hz highpass, and a 30 Hz lowpass second-order Butterworth filter. Just like the previous assignment, I used filtfilt to filter the data, ***but if I wanted to implement this online I would have to just use filt.*** For readibility:
- Filter.......: Butterworth, noncausal filtfilt
- Order......: 2nd
- High pass: 0.1 Hz
- Low pass.: 30 Hz

**Step 2: Determine our epoch, baseline, and window of focus**<br>
For the epoch, I decided to use 0ms (stimulus onset) to 500ms after the stimulus onset. This epoch length is effective since there is likely no relevant information after 500ms, and the P300 plot provided by the author seems to agree with this. For the baseline, I chose 0ms to 100ms after stimulus onset. This baseline window was chosen because every epoch contains information from the previous stimulus due to the small amount of time between stimuli. For the window we will select our ERP from, I chose 250ms after the stimulus to 450ms after the stimulus. This choice is consistent with the theoretical expected onset of the P300, and minimizes signals from previous and subsequent stimuli. For readibility:
- Epoch...: 0 - 500ms
- Baseline: 0 - 100ms
- ERP.......: 250 - 450ms

**Step 3: Determine how many signals to extract**<br>
There is a lot of mutual information in our data, and it would not be productive to export all of it, specifically because it would take more time to train/cross-validate our model. With this in mind, I decided to ten-fold sub-sample the temporal data, while preseving all 64 channels. I wanted to remain spatially agnostic with these data, although I could have reduced the amount of data by selecting only central + occipital electrodes, for example. With our sub-sampling, we acquire 5 samples per epoch, for all 64 channels, i.e. we obtain a *5 x 64* feature vector for every stimulus. For readibility:
- Sub-sampling..........: 10-fold
- Samples per epoch.: 5
- Number of channels: 64

The following is a visual explanation of the baseline, ERP window selection, and sub-sampling routine (on a single channel, using the figure provided by the author):

<img src="img1.png">

**Step 4: Extract the signals, and markers/labels**<br>
After deciding on everything, I extracted the signals and markers of interest. The exact details of this can be found in the MATLAB syntax. I saved the signals in a *5 x 64 x n_stimuli* numpy array, using a helpful function I found online. I saved the labels and markers of the training data in a *n_stimuli x 2* CSV, and the markers of the test data in an *n_stimuli x 1*  CSV. For readibility:
- Train signals...............: 'train_df.npy' (5 x 64 x n_stimuli)
- Test signals................: 'test_df.npy'  (5 x 64 x n_stimuli)
- Train markers + labels: 'train_la.csv' (n_stimuli x 2)
- Test markers...............: 'test_la.csv'  (n_stimuli x 1)

***

Now that we understand how our data were pre-processed and extracted, let's load it all in!

In [ ]:
# Load in train signals and markers + labels
X_train = np.load('train_df.npy')
markers_train = np.genfromtxt('train_la.csv', delimiter=',').astype(int) # also contains labels in first column
y = markers_train[:, 0] # let's only save the labels

# Let's also load in our test signals and markers
X_test = np.load('test_df.npy')
markers_test = np.genfromtxt('test_la.csv', delimiter=',').astype(int)

# Let's print out the shapes of our data
print('X_train shape is: ' + str(X_train.shape))
print('y shape is......: ' + str(y.shape))
print('X_test shape is.: ' + str(X_test.shape))

As we can see, the dimensions of *X_train* are 5 x 64 x 7561. We have 5 samples per observation (since we 10-times subsample a window of ~50 samples), 64 channels, and 7561 unique observations. Similarly, we have 5580 unique observations for our test data. We must restructure our data so that they can be passed into our classifier. Luckily, our 5 x 64 features can be flattened, and we can subsequently transpose our matrix so that the first dimension matches our label vector (this is the format sklearn expects). We will do the same flattening and transposing on our test data

In [ ]:
# Since our X is 3D, we must flatten our data. We will then transpose it for sklearn
X_train = X_train.reshape(-1, X_train.shape[-1]).T
X_test = X_test.reshape(-1, X_test.shape[-1]).T

# Let's print out the new shape
print('X_train shape is now: ' + str(X_train.shape))
print('X_test  shape is now: ' + str(X_test.shape))

Now that our data are correctly formatted, we can train a model! We will first choose our classifier. One standard choice for classifying P300 data is via linear discriminant analysis (LDA) which has been discussed in class. sklearn supports LDA (and its quadratic counterpart QDA) which makes the process rather simple. 
<br><br>
We will be comparing two different solvers for our classifier.  The first uses eigenvalue composition which can be computationally expensive for high-dimensional covariance matrices.  The second uses least-squares, which tends to be more efficient.
<br><br>
We will be using shrinkage, which is important considering how much data we have, and we will determine this parameter automatically.<br>
See: https://scikit-learn.org/stable/auto_examples/classification/plot_lda.html#sphx-glr-auto-examples-classification-plot-lda-py

In [ ]:
# Train our classifier
clf_eigen = LinearDiscriminantAnalysis(solver = 'eigen', shrinkage = 'auto').fit(X_train, y)
clf_lsqrs = LinearDiscriminantAnalysis(solver = 'lsqr',  shrinkage = 'auto').fit(X_train, y)

Let's determine which solver is performing best by using 5-fold cross validation<br>
***Note: This will take some time to run***

In [ ]:
# Let's do 5-fold cross validation to approximate which solver is better
score_eigen = cross_val_score(clf_eigen, X_train, y, cv = 5)
score_lsqrs = cross_val_score(clf_lsqrs, X_train, y, cv = 5)

# We will print out the mean score for each
print("solver = eigen accuracy: " + str(np.mean(score_eigen)))
print("solver = lsqr  accuracy: " + str(np.mean(score_lsqrs)))

Please note that this gives us only a general insight into the performance of our models. As mentioned in class, calculating AUC of ROC plots would be more informative. Since the *lsqr* solver seems to be more accurate, we will select it as our classifier, and use it to make predictions on our test dataset

In [ ]:
# Set clf to our best performing solver
clf = clf_lsqrs

# Make predictions on our dataset 
conf = clf.decision_function(X_test) # predicted confidence score
pred = clf.predict(X_test)           # predicted label (we won't actually use this)

At this stage, we have trained a model, and used it to make predictions on our test data. We will be using the confidence score of each classification to determine which character the subject was attending to. The larger the score, the more confident the classification was.
<br><br>
First, let's create a numpy array to mimic the character matrix described in the data description, in order to efficiently determine which character our classifier predicted
<img src="img2.png">

In [ ]:
# Here are the translations for the labels:
# Columns 0-5 -> Labels 1-6
# Rows 0-5 -> Labels 7-12
# Let's create this matrix
#                         1    2    3    4    5    6
char_matrix = np.array([['A', 'B', 'C', 'D', 'E', 'F'], # 7
                        ['G', 'H', 'I', 'J', 'K', 'L'], # 8
                        ['M', 'N', 'O', 'P', 'Q', 'R'], # 9
                        ['S', 'T', 'U', 'V', 'W', 'X'], # 10
                        ['Y', 'Z', '1', '2', '3', '4'], # 11
                        ['5', '6', '7', '8', '9', '0']])# 12 

We know that for each character, each row and column was flashed once, for a total of 12 flashes per block and this was repeated 15 times. This means that the first 12x15 data points correspond to the first character, the 2nd 12x15 data point correspond to the second character, etc.
<br><br>
Let's examine the first character from the test data

In [ ]:
# Define some useful variables
char_breaks = 15 * 12 # number of stimuli per character
c = 1;                # which character we want

# Grab our first character's marker, prediction, and confidence
char1_mrks = markers_test[char_breaks*(c-1):char_breaks*c]
char1_pred = pred[char_breaks*(c-1):char_breaks*c]
char1_conf = conf[char_breaks*(c-1):char_breaks*c]

# Let's merge these
char1 = np.array([char1_mrks, char1_pred, char1_conf]).T

We now have an array *char1* which contains 180 stimuli, their marker (which row/col), their prediction (0, 1 *unused*), and the confidence of the prediction (float). Let's sort our char1 by confidence in order to determine which row and column we should index

In [ ]:
# Alright, now let's sort them by the 3rd column, the confidence
# Nice 1-liner provided by https://stackoverflow.com/questions/2828059/sorting-arrays-in-numpy-by-column
char1_sort = char1[char1[:,2].argsort()[::-1]]

# Let's print out the first 5 results
char1_sort[0:5, :]

Our highest confidence is the marker '7' which corresponds to the first row. <br>
Our second highest confidence is the marker '6' which corresponds to the last column. <br>
We *could* simply take the highest column and row and consider that the prediction, however, it's possible that those values are outliers. In order to combat this, we will take the mean of every marker's confidence, and use the row and column with the **highest average confidence** to determine our character

In [ ]:
# Now we will store the average confidence value for every row/column
x = np.zeros(shape = (12, 1))
for i in range(0, x.shape[0]):
    x[i] = np.mean(char1_sort[char1_sort[:, 0] == i + 1, 2])
    
# Now let's print our character
row = np.argmax(x[6:])
col = np.argmax(x[0:6])
our_char = char_matrix[row, col]
print('Our character is: ' + our_char)

In [ ]:
# Let's quickly visualize the average confidence scores with a bar plot
plt.figure()
ticks = np.array(range(1, x.shape[0] + 1, 1))
plt.bar(ticks, np.ravel(x), tick_label = ticks)
plt.show()

Excellent! Our classifier predicts that the first character the subject attended to was the letter 'F'. Let's now use this logic and apply it iteratively to each character in the test data, and print out every character the user attended to:

In [ ]:
# Main loop: Iterate through test data by chunking every 15x12 trials per character
#  Nest 1: Calculate mean of character
char_breaks = 15*12
output_string = ''
for c in range(1, (X_test.shape[0] / char_breaks) + 1):
    # Lets break up test into char 'c'
    temp_mrks = markers_test[char_breaks * (c - 1):char_breaks * c]
    temp_pred = pred[char_breaks * (c - 1):char_breaks * c]
    temp_conf = conf[char_breaks * (c - 1):char_breaks * c]

    # Let's merge these and sort by highest confidence
    temp_char = np.array([temp_mrks, temp_pred, temp_conf]).T
    temp_char = temp_char[temp_char[:, 2].argsort()[::-1]]
    
    # Create our averages
    x = np.zeros(shape = (12, 1))
    for i in range(0, x.shape[0]):
        x[i, 0] = np.mean(temp_char[temp_char[:, 0] == i + 1, 2])
        
    # Now let's print our character
    row = np.argmax(x[6:,  0])
    col = np.argmax(x[0:6, 0])
    our_char = char_matrix[row, col]
    output_string += our_char + ' '
    
# Print our result
print(output_string)

Since this is a competition dataset, we were obviously not given the labels to our test set. However, since this competition is very old (2002), the results are public!
<br><br>
5 teams got first place in this competition, each with test character 100% accuracy. The added challenge was to minimize the amount of flashes each character was required, but for simplicity we will not do this.
<br><br>
Since all of these teams were able to determine the desired characters with 100% accuracy, we can look at one of their write-ups to determine the true labels of the test set.<br>
The approach we took for this assignment was influenced by: http://www.bbci.de/competition/ii/results/tax_iib_desc.pdf
<br>
***
***Question 6:*** What precentage of our test characters were correctly classified?
***